In [54]:
import pandas as pd 
import numpy as np
from qiime2 import Artifact
import os

Import of the final sample dataset obtained in the EDA notebook: 

In [55]:
data = pd.read_csv('data_final_groups2.csv',dtype = {"sample_name": "str"},index_col=0)
data

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency,group
0,10317.000010863,No,No,No,No,20s,25.0,25.0,Yes,Rarely (a few times/month),...,Never,Occasionally (1-2 times/week),Rarely (a few times/month),NaN,Remained stable,52,kilograms,Regularly (3-5 times/week),Not provided,Treatment
1,10317.000022229,"Diagnosed by a medical professional (doctor, p...",No,Yes,No,60s,63.0,63.0,Yes,Daily,...,Rarely (a few times/month),Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,106,kilograms,Rarely (less than once/week),Rarely (less than once/week),Treatment
2,10317.000022230,No,No,No,No,50s,58.0,58.0,Yes,Daily,...,Occasionally (1-2 times/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week),NaN,Remained stable,61,kilograms,Rarely (less than once/week),Rarely (less than once/week),Treatment
3,10317.000023139,No,No,No,No,60s,69.0,69.0,Yes,Rarely (a few times/month),...,Regularly (3-5 times/week),Daily,Never,NaN,Remained stable,63,kilograms,Regularly (3-5 times/week),Regularly (3-5 times/week),Treatment
4,10317.000023612,"Diagnosed by a medical professional (doctor, p...",No,No,No,60s,64.0,64.0,Yes,Occasionally (1-2 times/week),...,Never,Rarely (a few times/month),Never,NaN,Decreased more than 10 pounds,105,kilograms,Occasionally (1-2 times/week),Never,Treatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,10317.000067547,No,No,Yes,No,20s,24.0,24.0,Yes,Rarely (a few times/month),...,Never,Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,43,kilograms,Rarely (less than once/week),Occasionally (1-2 times/week),Control
855,10317.000067548,No,No,Yes,No,20s,24.0,24.0,Yes,Rarely (a few times/month),...,Never,Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,43,kilograms,Rarely (less than once/week),Occasionally (1-2 times/week),Control
856,10317.000067557,No,No,No,No,20s,26.0,26.0,Yes,Occasionally (1-2 times/week),...,Never,Never,Never,NaN,Remained stable,58,kilograms,Occasionally (1-2 times/week),Occasionally (1-2 times/week),Control
857,10317.000067558,No,No,No,No,20s,26.0,26.0,Yes,Occasionally (1-2 times/week),...,Never,Never,Never,NaN,Remained stable,58,kilograms,Occasionally (1-2 times/week),Occasionally (1-2 times/week),Control


In [56]:
data.specialized_diet_exclude_refined_sugars.value_counts()

No     500
Yes    359
Name: specialized_diet_exclude_refined_sugars, dtype: int64

## Metadata file creation

In [57]:
metadata = data.sample_name.to_frame()

In [58]:
metadata = metadata.rename(columns={'sample_name': 'sample-id'})

In [59]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
metadata['sex'] = data['sex']
metadata['age_cat'] = data['age_cat']
metadata['refined_sugars'] = data['specialized_diet_exclude_refined_sugars']
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000010863,female,20s,Yes
1,10317.000022229,male,60s,Yes
2,10317.000022230,female,50s,Yes
3,10317.000023139,female,60s,Yes
4,10317.000023612,male,60s,Yes
...,...,...,...,...
854,10317.000067547,female,20s,No
855,10317.000067548,female,20s,No
856,10317.000067557,female,20s,No
857,10317.000067558,female,20s,No


In [60]:
metadata['sample-id'].to_csv("ids.txt",header=None,index=None)

In [61]:
metadata.refined_sugars.value_counts()

No     500
Yes    359
Name: refined_sugars, dtype: int64

In [62]:
metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)

In [63]:
from qiime2 import Metadata
sample_metadata = Metadata.load(os.getcwd()+'/sample_metadata.tsv')

Import Fasta 99% otus containing Taxonomy

In [64]:
!qiime tools import --type 'FeatureData[Sequence]' --input-path gg_13_8_otus/rep_set/99_otus.fasta \
    --output-path rep_set.qza

Imported gg_13_8_otus/rep_set/99_otus.fasta as DNASequencesDirectoryFormat to rep_set.qza


Getting otus from sample ids with biom:

In [15]:
!export ctx=Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-5c6506

In [ ]:
!redbiom fetch samples --from ids.txt --context $ctx --output otu_table.biom

In [65]:
!biom convert -i otu_table.biom -o otu_table.tsv --to-tsv 

In [68]:
meta_table = pd.read_csv("sample_metadata.tsv",sep="\t",dtype = {"sample-id": "str"})
meta_table = meta_table.sort_values(by="sample-id").reset_index(drop=True)

In [69]:
!qiime tools import \
  --input-path otu_table.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table.qza

Imported otu_table.biom as BIOMV210Format to feature_table.qza


In [71]:
from qiime2 import Artifact
otu_table = Artifact.load("feature_table.qza")
rep_set = Artifact.load("rep_set.qza")

In [72]:
otu_index = otu_table.view(pd.DataFrame).index.sort_values()

In [73]:
ot_df = otu_table.view(pd.DataFrame) 

In [75]:
non_match = []
for i in range(meta_table.shape[0]):
    result = [k for k in otu_index if k.startswith(meta_table["sample-id"][i])]
    if len(result) == 0:
        non_match.append(meta_table["sample-id"][i] )
    else: 
        meta_table["sample-id"][i] = meta_table["sample-id"][i] + result[0][-6:]

In [76]:
meta_table_filt = meta_table.loc[~meta_table["sample-id"].isin(non_match)] 
    

In [90]:
meta_table_filt

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000003509.48742,male,30s,No
3,10317.000004057.42298,male,60s,No
4,10317.000004936.25853,male,50s,No
...,...,...,...,...
847,10317.000110309.76952,male,30s,Yes
848,10317.000110416.76952,male,50s,Yes
849,10317.000112912.76952,female,50s,Yes
850,10317.000112984.81168,male,50s,Yes


In [77]:
meta_table_filt.to_csv("sample_metadata_filt.tsv",sep="\t",index=None)

In [78]:
ot_df_filt = ot_df.loc[ot_df.index.isin(meta_table_filt["sample-id"])]

In [79]:
ot_df_filt.to_csv("otu_table_filt.tsv",sep="\t")

In [80]:
!biom convert -i otu_table_filt.tsv -o otu_table_filt.biom --table-type="OTU table" --to-hdf5

In [81]:
!qiime feature-table filter-seqs --i-data rep_set.qza --m-metadata-file otu_table.tsv --o-filtered-data rep_set_filt.qza

Saved FeatureData[Sequence] to: rep_set_filt.qza


In [135]:
!biom convert -i otu_table_reorder.tsv -o otu_table_reorder.biom --table-type="OTU table" --to-hdf5

In [86]:
otu_df = otu_table.view(pd.DataFrame)

OTU filtering

In [89]:
otu_df

,208901,1117675,193418,4341194,4468384,849535,113919,430388,4404392,180397,...,369390,182142,287755,2792167,1902400,188454,782984,214651,4368600,4463532
10317.000001166.25853,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
10317.000003100.69016,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0
10317.000003509.48742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,44.0
10317.000004057.42298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000004936.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000110309.76952,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10317.000110416.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
10317.000112912.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000112984.81168,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,112.0


In [88]:
otu_df = otu_df.loc[otu_df.index.isin(meta_table_filt["sample-id"])].sort_index()

In [91]:
otu_df.to_csv("otu_match.tsv", sep='\t')

In [85]:
!biom convert -i otu_match.tsv -o otu_match.biom --table-type="OTU table" --to-hdf5

Usage: biom convert [OPTIONS]
Try 'biom convert -h' for help.

Error: Invalid value for '-i' / '--input-fp': File 'otu_match.tsv' does not exist.


In [104]:
!qiime tools import \
  --input-path otu_match.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table_match.qza

Imported otu_match.biom as BIOMV210Format to feature_table.qza


In [113]:
!qiime feature-table transpose --i-table feature_table_match.qza --o-transposed-feature-table feature_table_match.qza

Saved FeatureTable[Frequency] to: feature_table.qza


In [ ]:
from qiime2.plugins import alignment, phylogeny
mafft_alignment = alignment.methods.mafft(rep_set)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree = phylogeny.methods.midpoint_root(unrooted_tree.tree)


In [28]:
rooted_tree.rooted_tree.save("rooted_tree.qza")

'rooted_tree.qza'

In [29]:
rooted_tree = Artifact.load("rooted_tree.qza")

In [114]:
otu_table = Artifact.load("feature_table.qza")

In [106]:
output_viz = feature_table.visualizers.summarize(otu_table)
output_viz.visualization.save("otu_first_analysis.qzv")

'otu_first_analysis.qzv'

In [107]:
from qiime2 import Metadata
metadata = Metadata.load("sample_metadata.tsv")

In [94]:
metadata = pd.read_csv('sample_metadata.tsv', sep='\t')
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
354,10317.000109791.74729,male,20s,Yes
355,10317.000109807.76952,female,40s,Yes
356,10317.000110177.74729,male,30s,Yes
357,10317.000112912.76952,female,50s,Yes


In [89]:
metadata["sample-id"]

0      10317.000001166.25853
1      10317.000003100.69016
2      10317.000004936.25853
3      10317.000010862.25853
4      10317.000012018.56079
               ...          
354    10317.000109791.74729
355    10317.000109807.76952
356    10317.000110177.74729
357    10317.000112912.76952
358    10317.000113026.81168
Name: sample-id, Length: 359, dtype: object

In [83]:
otu_table = Artifact.load("feature_table.qza")
otu_table.view(pd.DataFrame)

,208901,1117675,193418,4341194,4468384,849535,113919,430388,4404392,180397,...,369390,182142,287755,2792167,1902400,188454,782984,214651,4368600,4463532
10317.000010863.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,61.0
10317.000022229.45578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000022230.45578,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
10317.000023139.25853,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10317.000023612.30606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000067548.26479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
10317.000067557.26479,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,30.0
10317.000067558.26479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000067558.30606,0.0,0.0,0.0,0.0,11.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0


In [84]:
import biom

b = biom.load_table("otu_table_filt.biom")
b

797 x 8790 <class 'biom.table.Table'> with 393939 nonzero entries (5% dense)

In [91]:
np.sum(otu_table.view(pd.DataFrame).columns == metadata["sample-id"])

359

In [116]:
from qiime2.plugins import diversity
core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = otu_table,
                                                             phylogeny = rooted_tree,
                                                             sampling_depth = 8200,
                                                             metadata = metadata)

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.3931712073149837 and the largest is 24.638091594948897.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/gr887277/data/c1d88312-0f0a-4593-9fd0-dddee5ee4bdd/data/feature-table.biom -t /tmp/qiime2/gr887277/data/d9692bf3-0edd-4885-936f-5cb62f3b81a8/data/tree.nwk -o /tmp/q2-AlphaDiversityFormat-sily2v3c

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/gr887277/data/c1d88312-0f0a-4593-9fd0-dddee5ee4bdd/data/feature-table.biom -t /tmp/qiime2/gr887277/data/d9692bf3-0edd-4885-936f-5cb62f3b81a8/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-yfrd5ihu

Running external command line application. This may print messages to stdout and/or stderr.
T

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0013194195884127862 and the largest is 5.979319453635784.
  warn(
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. Th

In [68]:
import pandas as pd

In [117]:
core_metrics.rarefied_table.view(pd.DataFrame)

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,4356697,213762,179905,4324040,333726,199147,1808779,173916,435264,325586
10317.000001166.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000003100.69016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000010862.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000012018.56079,0.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000013727.69016,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000109095.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109791.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109807.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000110177.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
core_metrics_rarefied_table_df = core_metrics.rarefied_table.view(pd.DataFrame).reset_index().melt(id_vars=['index'],
                                                          var_name="feature", 
                                                          value_name="Value")
core_metrics_rarefied_table_df.to_csv('core_metrics_rarefied_table_df.csv')
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [119]:
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [121]:
observed_otus_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.observed_otus_vector,
                                                                                     metadata)
observed_otus_vector_group_significance.visualization

AttributeError: 'Results' object has no attribute 'observed_otus_vector'

In [122]:
shannon_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.shannon_vector,
                                                                         metadata)
shannon_vector_group_significance.visualization.save("shannon.qzv")

'shannon.qzv'

In [123]:
faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector,
                                                                         metadata)
faith_pd_group_significance.visualization.save("faith_pd.qzv")

'faith_pd.qzv'

In [124]:
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector,
                                                                         metadata)
evenness_group_significance.visualization.save("evenness_group.qzv")

'evenness_group.qzv'

In [125]:
bray_curtis_distance_significance = diversity.actions.beta_group_significance(core_metrics.bray_curtis_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
bray_curtis_distance_significance.visualization.save("bray_curtis.qzv")

'bray_curtis.qzv'

In [126]:
jaccard_distance_significance = diversity.actions.beta_group_significance(core_metrics.jaccard_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
jaccard_distance_significance.visualization.save("jaccard_beta.qzv")

'jaccard_beta.qzv'

In [132]:
unweighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
unweighted_unifrac_significance.visualization.save("unweighted_unifrac.qzv")

'unweighted_unifrac.qzv'

In [131]:
weighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.weighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
weighted_unifrac_significance.visualization.save("weighted_unifrac.qzv")

'weighted_unifrac.qzv'

In [135]:
from qiime2.plugins import emperor
emperor_plot = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results,
                                        metadata)
                                        #custom_axes=['DaysSinceExperimentStart'])
emperor_plot.visualization.save('emperor.qzv')

'emperor.qzv'